In [2]:
# Imports
import gymnasium as gym
from typing import Callable

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.vec_env import SubprocVecEnv
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.vec_env import VecMonitor
import torch as th
from stable_baselines3.common.callbacks import EvalCallback

import pickle

# Import Our environment
from dev_env import tradingEng



c:\Users\osc16\miniconda3\envs\d2d\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [3]:
# Load Paths
with open("ZeroCorrFrs1Half","rb") as fp:
    paths1 = pickle.load(fp)

# Load Paths
with open("ZeroCorrFrs2Half","rb") as fp:
    paths1 = paths1 + pickle.load(fp)

with open("ZeroCorrSnd1Half","rb") as fp:
    paths2 = pickle.load(fp)

with open("ZeroCorrSnd2Half","rb") as fp:
    paths2 = paths2 + pickle.load(fp)

with open("ZeroCorrTest","rb") as fp:
    paths_ev = pickle.load(fp)


In [4]:

## LR schedule
def linear_schedule(initial_value: float) -> Callable[[float], float]:
    """
    Linear learning rate schedule.

    :param initial_value: Initial learning rate.
    :return: schedule that computes
      current learning rate depending on remaining progress
    """
    def func(progress_remaining: float) -> float:
        """
        Progress will decrease from 1 (beginning) to 0.

        :param progress_remaining:
        :return: current learning rate
        """
        return progress_remaining * initial_value

    return func
policy_kwargs = dict(activation_fn=th.nn.LeakyReLU,
                     net_arch=dict(pi=[512,512,256,128,64,64,64,64,36,18], vf=[512,512,256,128,64,64,64,64,36,18], optimizers_class = th.optim.Adam, log_std_init = 0.0005)) #


In [ ]:
# Horizons

# Default Big Batch
envs = VecMonitor(DummyVecEnv([
    lambda: tradingEng(paths1,action = 'small-More-Trust', obs = 'xs'),
    lambda: tradingEng(paths2,action = 'small-More-Trust', obs = 'xs')
]),filename='eval_logs-Train')
ev_env = VecMonitor(DummyVecEnv([
    lambda: tradingEng(paths_ev,action = 'small-More-Trust', obs = 'xs'),
]))

eval_callback = EvalCallback(
    ev_env,
    best_model_save_path='./logs/best_modelB',
    log_path='./logs/eval_logs/evB',
    eval_freq=252*20*5,
    deterministic=True,
    render=False,
    verbose = True,
    n_eval_episodes = 16
)

model = PPO("MlpPolicy", envs, batch_size = 252*10*5, learning_rate=linear_schedule(0.005), policy_kwargs=policy_kwargs, n_steps=252*20*5, normalize_advantage=True, gamma = 0.9, verbose = 1) 

model.learn(total_timesteps=3e6, log_interval=2, callback=eval_callback) 

# Default Lower LR
envs = VecMonitor(DummyVecEnv([
    lambda: tradingEng(paths1,action = 'small-More-Trust', obs = 'xs'),
    lambda: tradingEng(paths2,action = 'small-More-Trust', obs = 'xs')
]),filename='eval_logs-Train')
ev_env = VecMonitor(DummyVecEnv([
    lambda: tradingEng(paths_ev,action = 'small-More-Trust', obs = 'xs'),
]))

eval_callback = EvalCallback(
    ev_env,
    best_model_save_path='./logs/best_modelLLR',
    log_path='./logs/eval_logs/evLLR',
    eval_freq=252*20*5,
    deterministic=True,
    render=False,
    verbose = True,
    n_eval_episodes = 16
)

model = PPO("MlpPolicy", envs, batch_size = 252*2*5, learning_rate=linear_schedule(0.001), policy_kwargs=policy_kwargs, n_steps=252*4*5, normalize_advantage=True, gamma = 0.9, verbose = 1) 

model.learn(total_timesteps=3e6, log_interval=2, callback=eval_callback) 

# Default Both
envs = VecMonitor(DummyVecEnv([
    lambda: tradingEng(paths1,action = 'small-More-Trust', obs = 'xs'),
    lambda: tradingEng(paths2,action = 'small-More-Trust', obs = 'xs')
]),filename='eval_logs-Train')
ev_env = VecMonitor(DummyVecEnv([
    lambda: tradingEng(paths_ev,action = 'small-More-Trust', obs = 'xs'),
]))

eval_callback = EvalCallback(
    ev_env,
    best_model_save_path='./logs/best_modelLLRB',
    log_path='./logs/eval_logs/evLLRB',
    eval_freq=252*20*5,
    deterministic=True,
    render=False,
    verbose = True,
    n_eval_episodes = 16
)

model = PPO("MlpPolicy", envs, batch_size = 252*10*5, learning_rate=linear_schedule(0.001), policy_kwargs=policy_kwargs, n_steps=252*20*5, normalize_advantage=True, gamma = 0.9, verbose = 1) 

model.learn(total_timesteps=3e6, log_interval=2, callback=eval_callback) 

## 1 yr
#envs = VecMonitor(DummyVecEnv([
#    lambda: tradingEng(paths1,action = 'small-More-Trust', obs = 'xs', resetdate=1.0),
#    lambda: tradingEng(paths2,action = 'small-More-Trust', obs = 'xs',resetdate=1.0)
#]),filename='logs1-Train')
#ev_env = VecMonitor(DummyVecEnv([
#    lambda: tradingEng(paths_ev,action = 'small-More-Trust', obs = 'xs',resetdate=1.0),
#]))
#
#eval_callback = EvalCallback(
#    ev_env,
#    best_model_save_path='./logs/best_model1',
#    log_path='./logs/eval_logs1/ev',
#    eval_freq=252*8*1,
#     deterministic=True,
#    render=False,
#    verbose = True,
#    n_eval_episodes = 8
#)

# Instantiate the agent
#model = PPO("MlpPolicy", envs, batch_size = 252*2*1, learning_rate=linear_schedule(0.005), policy_kwargs=policy_kwargs, n_steps=252*4*1, normalize_advantage=True, gamma = 0.9, verbose = 1) 

#model.learn(total_timesteps=3e6, log_interval=2, callback=eval_callback) 

# 20 yr
#envs = VecMonitor(DummyVecEnv([
#    lambda: tradingEng(paths1,action = 'small-More-Trust', obs = 'xs',resetdate=20.0),
#    lambda: tradingEng(paths2,action = 'small-More-Trust', obs = 'xs',resetdate=20.0)
#]),filename='/logs/eval_logs20/Train')
#ev_env = VecMonitor(DummyVecEnv([
#    lambda: tradingEng(paths_ev,action = 'small-More-Trust', obs = 'xs',resetdate=20.0),
#]))

#eval_callback = EvalCallback(
#    ev_env,
#    best_model_save_path='./logs/best_model20',
#    log_path='./logs/eval_logs20/ev',
#    eval_freq=252*8*20,
#    deterministic=True,
#    render=False,
#    verbose = True,
#    n_eval_episodes = 8
#)
#
#model = PPO("MlpPolicy", envs, batch_size = 252*2*20, learning_rate=linear_schedule(0.005), policy_kwargs=policy_kwargs, n_steps=252*4*20, normalize_advantage=True, gamma = 0.9, verbose = 1) #

#model.learn(total_timesteps=3e6, log_interval=2, callback=eval_callback) 

Using cpu device
Eval num_timesteps=20160, episode_reward=-0.11 +/- 0.03
Episode length: 1259.00 +/- 0.00
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 1.26e+03    |
|    mean_reward          | -0.115      |
| time/                   |             |
|    total_timesteps      | 20160       |
| train/                  |             |
|    approx_kl            | 0.006457979 |
|    clip_fraction        | 0.0451      |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.78       |
|    explained_variance   | 0.0325      |
|    learning_rate        | 0.00498     |
|    loss                 | -0.0012     |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.00148    |
|    std                  | 0.966       |
|    value_loss           | 0.00256     |
-----------------------------------------
New best mean reward!
---------------------------------
| rollout/           |          |
|    e

In [ ]:
# Big observation
envs = VecMonitor(DummyVecEnv([
    lambda: tradingEng(paths1,action = 'small-More-Trust', obs = 'small'),
    lambda: tradingEng(paths2,action = 'small-More-Trust', obs = 'small')
]),filename='logsBigO-train')
ev_env = VecMonitor(DummyVecEnv([
    lambda: tradingEng(paths_ev,action = 'small-More-Trust', obs = 'small'),
]))

eval_callback = EvalCallback(
    ev_env,
    best_model_save_path='./logs/best_modelBigObs',
    log_path='./logs/eval_logsBigO/ev',
    eval_freq=252*8*5,
    deterministic=True,
    render=False,
    verbose = True,
    n_eval_episodes = 8
)
model = PPO("MlpPolicy", envs, batch_size = 252*2*5, learning_rate=linear_schedule(0.005), policy_kwargs=policy_kwargs, n_steps=252*4*5, normalize_advantage=True, gamma = 0.9, verbose = 1) 

model.learn(total_timesteps=3e6, log_interval=2, callback=eval_callback) 

# Auto Encoder Observation
envs = VecMonitor(DummyVecEnv([
    lambda: tradingEng(paths1,action = 'small-More-Trust', obs = 'auto'),
    lambda: tradingEng(paths2,action = 'small-More-Trust', obs = 'auto')
]),filename='logsBigO-train')
ev_env = VecMonitor(DummyVecEnv([
    lambda: tradingEng(paths_ev,action = 'small-More-Trust', obs = 'auto'),
]))

eval_callback = EvalCallback(
    ev_env,
    best_model_save_path='./logs/best_modelAutoObs',
    log_path='./logs/eval_logsAutoO/ev',
    eval_freq=252*8*5,
    deterministic=True,
    render=False,
    verbose = True,
    n_eval_episodes = 8
)
model = PPO("MlpPolicy", envs, batch_size = 252*2*5, learning_rate=linear_schedule(0.005), policy_kwargs=policy_kwargs, n_steps=252*4*5, normalize_advantage=True, gamma = 0.9, verbose = 1) 

model.learn(total_timesteps=3e6, log_interval=2, callback=eval_callback) 





Using cpu device
Eval num_timesteps=20160, episode_reward=-0.08 +/- 0.03
Episode length: 1259.00 +/- 0.00
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 1.26e+03    |
|    mean_reward          | -0.0821     |
| time/                   |             |
|    total_timesteps      | 20160       |
| train/                  |             |
|    approx_kl            | 0.010723251 |
|    clip_fraction        | 0.0542      |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.77       |
|    explained_variance   | -1.45       |
|    learning_rate        | 0.00498     |
|    loss                 | -0.00504    |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.00254    |
|    std                  | 0.955       |
|    value_loss           | 0.0433      |
-----------------------------------------
New best mean reward!
---------------------------------
| rollout/           |          |
|    e

In [ ]:
# Big Action
envs = VecMonitor(DummyVecEnv([
    lambda: tradingEng(paths1,action = 'big', obs = 'xs'),
    lambda: tradingEng(paths2,action = 'big', obs = 'xs')
]),filename='logsBigA-Train')
ev_env = VecMonitor(DummyVecEnv([
    lambda: tradingEng(paths_ev,action = 'big', obs = 'xs'),
]))

eval_callback = EvalCallback(
    ev_env,
    best_model_save_path='./logs/best_modelBigAct',
    log_path='./eval_logsBigA/ev',
    eval_freq=252*8*5,
    deterministic=True,
    render=False,
    verbose = True,
    n_eval_episodes = 8
)
model = PPO("MlpPolicy", envs, batch_size = 252*2*5, learning_rate=linear_schedule(0.005), policy_kwargs=policy_kwargs, n_steps=252*4*5, normalize_advantage=True, gamma = 0.9, verbose = 1) 

model.learn(total_timesteps=3e6, log_interval=2, callback=eval_callback) 



Using cpu device
Eval num_timesteps=20160, episode_reward=-0.13 +/- 0.04
Episode length: 1259.00 +/- 0.00
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 1.26e+03    |
|    mean_reward          | -0.13       |
| time/                   |             |
|    total_timesteps      | 20160       |
| train/                  |             |
|    approx_kl            | 0.007813591 |
|    clip_fraction        | 0.109       |
|    clip_range           | 0.2         |
|    entropy_loss         | -56.8       |
|    explained_variance   | -0.0434     |
|    learning_rate        | 0.00498     |
|    loss                 | -0.00487    |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0052     |
|    std                  | 1           |
|    value_loss           | 0.000691    |
-----------------------------------------
New best mean reward!
---------------------------------
| rollout/           |          |
|    e

In [ ]:
# Reward l2
envs = VecMonitor(DummyVecEnv([
    lambda: tradingEng(paths1,action = 'small', obs = 'xs', reward = '2a'),
    lambda: tradingEng(paths2,action = 'small', obs = 'xs', reward = '2a')
]),filename='logsL2-train')

ev_env = VecMonitor(DummyVecEnv([
    lambda: tradingEng(paths_ev,action = 'small-More-Trust', obs = 'xs', reward = '2a'),
]))

eval_callback = EvalCallback(
    ev_env,
    best_model_save_path='./logs/best_modelL2',
    log_path='./logs/eval_logsL2/ev',
    eval_freq=252*8*5,
    deterministic=True,
    render=False,
    verbose = True,
    n_eval_episodes = 8
)

# Instantiate the agent
policy_kwargs = dict(activation_fn=th.nn.LeakyReLU,
                     net_arch=dict(pi=[512,512,256,128,64,64,64,64,36,18], vf=[512,512,256,128,64,64,64,64,36,18], optimizers_class = th.optim.Adam, log_std_init = 0.005)) #
model = PPO("MlpPolicy", envs, batch_size = 252*2*5, learning_rate=linear_schedule(0.005), policy_kwargs=policy_kwargs, n_steps=252*4*5, normalize_advantage=True, gamma = 0.9, verbose = 1) 

model.learn(total_timesteps=3e6, log_interval=2, callback=eval_callback) 

Using cpu device
Eval num_timesteps=20160, episode_reward=0.00 +/- 0.00
Episode length: 1259.00 +/- 0.00
------------------------------------------
| eval/                   |              |
|    mean_ep_length       | 1.26e+03     |
|    mean_reward          | 0            |
| time/                   |              |
|    total_timesteps      | 20160        |
| train/                  |              |
|    approx_kl            | 0.0012949443 |
|    clip_fraction        | 0.00668      |
|    clip_range           | 0.2          |
|    entropy_loss         | -2.85        |
|    explained_variance   | -3.21        |
|    learning_rate        | 0.00498      |
|    loss                 | 0.00071      |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.000504    |
|    std                  | 1            |
|    value_loss           | 0.00041      |
------------------------------------------
New best mean reward!
---------------------------------
| rollout/           |

In [ ]:
# Add ent
envs = VecMonitor(DummyVecEnv([
    lambda: tradingEng(paths1,action = 'small', obs = 'xs'),
    lambda: tradingEng(paths2,action = 'small', obs = 'xs')
]),filename='logs-TrainEnt')
ev_env = VecMonitor(DummyVecEnv([
    lambda: tradingEng(paths_ev,action = 'small', obs = 'xs'),
]))

eval_callback = EvalCallback(
    ev_env,
    best_model_save_path='./logs/best_modelEnt',
    log_path='./logs/eval_logs/evEnt',
    eval_freq=252*8*5,
    deterministic=True,
    render=False,
    verbose = True,
    n_eval_episodes = 8
)

model = PPO("MlpPolicy", envs, batch_size = 252*2*5, learning_rate=linear_schedule(0.005), policy_kwargs=policy_kwargs, n_steps=252*4*5, normalize_advantage=True, gamma = 0.9, verbose = 1,ent_coef=0.01) 

model.learn(total_timesteps=3e6, log_interval=2, callback=eval_callback) 


Using cpu device
Eval num_timesteps=20160, episode_reward=-0.08 +/- 0.02
Episode length: 1259.00 +/- 0.00
------------------------------------------
| eval/                   |              |
|    mean_ep_length       | 1.26e+03     |
|    mean_reward          | -0.0806      |
| time/                   |              |
|    total_timesteps      | 20160        |
| train/                  |              |
|    approx_kl            | 0.0042122398 |
|    clip_fraction        | 0.0248       |
|    clip_range           | 0.2          |
|    entropy_loss         | -2.8         |
|    explained_variance   | -0.417       |
|    learning_rate        | 0.00498      |
|    loss                 | -0.0274      |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.0015      |
|    std                  | 0.977        |
|    value_loss           | 0.000769     |
------------------------------------------
New best mean reward!
---------------------------------
| rollout/           

In [ ]:
# Correlations 0.7
#t = start_and_release(paths1,action='small-More-Trust', obs = 'auto')
with open("0.7Corr1Half.pkl","rb") as fp:
    paths1 = pickle.load(fp)

# Load Paths
with open("0.7Corr2Half.pkl","rb") as fp:
    paths2 = pickle.load(fp)

with open("0.7CorrTest.pkl","rb") as fp:
    paths_ev = pickle.load(fp)

envs = VecMonitor(DummyVecEnv([
    lambda: tradingEng(paths1,action = 'small', obs = 'xs'),
    lambda: tradingEng(paths2,action = 'small', obs = 'xs')
]),filename='logs07-train')

ev_env = VecMonitor(DummyVecEnv([
    lambda: tradingEng(paths_ev,action = 'small', obs = 'xs'),
]))

eval_callback = EvalCallback(
    ev_env,
    best_model_save_path='./logs/best_model07',
    log_path='./logs/eval_logs07',
    eval_freq=252*8*5*20,
    deterministic=True,
    render=False,
    verbose = True,
    n_eval_episodes = 8
)

# Instantiate the agent
policy_kwargs = dict(activation_fn=th.nn.LeakyReLU,
                     net_arch=dict(pi=[512,512,256,128,64,64,64,64,36,18], vf=[512,512,256,128,64,64,64,64,36,18], optimizers_class = th.optim.Adam, log_std_init = 0.005)) #
model = PPO("MlpPolicy", envs, batch_size = 252*4*5, learning_rate=linear_schedule(0.0015), policy_kwargs=policy_kwargs, n_steps=252*8*5, normalize_advantage=True, gamma = 0.9, verbose = 1) 

model.learn(total_timesteps=1e7, log_interval=2, callback=eval_callback) 

# Correlations -0.7
#t = start_and_release(paths1,action='small-More-Trust', obs = 'auto')
with open("NegCorrpt0.pkl","rb") as fp:
    paths1 = pickle.load(fp)
with open("NegCorrpt2.pkl","rb") as fp:
    paths1 = paths1 + pickle.load(fp)
# Load Paths
with open("NegCorrpt4.pkl","rb") as fp:
    paths2 = pickle.load(fp)
with open("NegCorrpt3.pkl","rb") as fp:
    paths2 = paths2 + pickle.load(fp)
with open("n0.7CorrTest.pkl","rb") as fp:
    paths_ev = pickle.load(fp)

envs = VecMonitor(DummyVecEnv([
    lambda: tradingEng(paths1,action = 'small', obs = 'xs'),
    lambda: tradingEng(paths2,action = 'small', obs = 'xs')
]),filename='logsm07-train')

ev_env = VecMonitor(DummyVecEnv([
    lambda: tradingEng(paths_ev,action = 'small', obs = 'xs'),
]))

eval_callback = EvalCallback(
    ev_env,
    best_model_save_path='./logs/best_modelm07',
    log_path='./logs/eval_logsm07',
    eval_freq=252*8*5*20,
    deterministic=True,
    render=False,
    verbose = True,
    n_eval_episodes = 8
)

# Instantiate the agent
policy_kwargs = dict(activation_fn=th.nn.LeakyReLU,
                     net_arch=dict(pi=[512,512,256,128,64,64,64,64,36,18], vf=[512,512,256,128,64,64,64,64,36,18], optimizers_class = th.optim.Adam, log_std_init = 0.005)) #
model = PPO("MlpPolicy", envs, batch_size = 252*4*5, learning_rate=linear_schedule(0.0015), policy_kwargs=policy_kwargs, n_steps=252*8*5, normalize_advantage=True, gamma = 0.9, verbose = 1) 

model.learn(total_timesteps=1e7, log_interval=2, callback=eval_callback) 



